In [1]:
from openai import OpenAI
import openai
import pandas as pd
client = OpenAI(api_key = 'OPENAI_API_KEY')

In [2]:
# Load the CSV file containing the input data
input_data = pd.read_csv("dataset/validated_CFA.csv")

In [3]:
los_data = pd.read_csv("dataset/Cleanedfinal_output.csv")

In [4]:
input_data.head()

,Name_of_the_topic,Year,Level,Introduction_Summary,Learning_Outcomes,Link_to_the_Summary_Page,Link_to_the_PDF_File,Summary
0,Time-Series Analysis,2024.0,2,"As financial analysts, we often use time-serie...",calculate and evaluate the predicted trend val...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN
1,Credit Analysis Models,2024.0,2,Credit analysis plays an important role in the...,"explain expected exposure, the loss given defa...",https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN
2,Introduction to Alternative Investments,2023.0,1,"In this section, we explain what alternative i...",describe types and categories of alternative i...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN
3,Credit Default Swaps,2024.0,2,Derivative instruments in which the underlying...,"describe credit default swaps (CDS), single-na...",https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN
4,Valuation of Contingent Claims,2024.0,2,A contingent claim is a derivative instrument ...,describe and interpret the binomial option val...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,NaN


In [5]:
# Filter data for the specified topics
topics = [
    "Big Data Projects",
    "Backtesting & Simulation",
    "Branded Image Link Added to Refresher Reading"
]
filtered_data = input_data[input_data['Name_of_the_topic'].isin(topics)]
filtered_data.head()

,Name_of_the_topic,Year,Level,Introduction_Summary,Learning_Outcomes,Link_to_the_Summary_Page,Link_to_the_PDF_File,Summary
123,Big Data Projects,2021.0,2,Big data (also referred to as alternative data...,state and explain steps in a data analysis pro...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,"Big data—defined as data with volume, velocity..."
141,Backtesting & Simulation,2021.0,2,This reading provides an overview of backtesti...,describe objectives in backtesting an investme...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...,"In this reading, we have discussed how to perf..."
142,Branded Image Link Added to Refresher Reading,NaN,1,The authors of this article derive their formu...,Make sure you can add a BrandedImageLink compo...,https://www.cfainstitute.org/membership/profes...,NaN,"In today's competitive business environment, e..."


In [6]:
list_learning = {}
# for i in filtered_data:
#     print(i)
#     list_learning[i['Name_of_the_topic']] = i["Learning_Outcomes"].split(";")
# list_learning



In [7]:
# Filter data for the specified topics

In [8]:

# Function to generate technical notes for each LOS using GPT-3.5 Turbo
def generate_technical_notes(los, Summary):
    prompt = "Use the knowledge from {} to answer folllow up questions".format(Summary)
    query = "givme me detaild summery on the topic {} from you knowledge base return the response in markdown format".format(los)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": query}
        ],
        temperature=0,
    )
    return response.choices[0].message.content

In [12]:
import os

for index, row in filtered_data.iterrows():
    summary_new = row["Summary"]
    topic = row['Name_of_the_topic']
    learning_outcomes = row["Learning_Outcomes"].split(";")
    print(learning_outcomes)
    
    for los in learning_outcomes:
        res = generate_technical_notes(los, summary_new)
        filename = f"{topic}_{los}"
        filename = filename[:240]+'.md'
        filepath = os.path.join("markdown", filename)  # Specify your directory
        with open(filepath, 'w') as file:
            file.write(res)



['state and explain steps in a data analysis project', ' describe objectives, steps, and examples of preparing and wrangling data', ' describe objectives, methods, and examples of data exploration', ' describe objectives, steps, and techniques in model training', ' describe preparing, wrangling, and exploring text-based data for financial forecasting', ' describe methods for extracting, selecting and engineering features from textual data', ' evaluate the fit of a machine learning algorithm.']
['describe objectives in backtesting an investment strategy', ' describe and contrast steps and procedures in backtesting an investment strategy', ' interpret metrics and visuals reported in a backtest of an investment strategy', ' identify problems in a backtest of an investment strategy', ' describe different ways to construct multifactor models', ' compare methods of modeling randomness', ' evaluate and interpret a scenario analysis', ' contrast Monte Carlo and historical simulation', ' explai

In [13]:
import os

def read_markdown_files(directory_path):
    notes = {}
    for filename in os.listdir(directory_path):
        if filename.endswith(".md"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                notes[filename] = file.read()
    return notes


In [14]:

import pprint

def get_embeddings(text):
    response = client.embeddings.create(
        input=[text],
        model="text-embedding-ada-002"  # Or any model that suits your needs
    )
    
    pprint.pprint(vars(response))

    # embedding = response['data'][0]['embedding']
    # embedding = response['data'][0].embedding
    embedding = response.data[0].embedding  # Adjust based on actual structure


    return embedding


In [15]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [16]:
import pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="PINECONE_KEY")

index_name = "technical-notes"
print(pc.list_indexes())
if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
 # Match the embedding size
index_pc = pc.Index(index_name)



{'indexes': [{'dimension': 384,
              'host': 'set-a-7sdwtci.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'set-a',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 384,
              'host': 'langchain-chatbot-7sdwtci.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain-chatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [20]:
directory_path = 'markdown'

# Read markdown files
notes = read_markdown_files(directory_path)

# Process each note
for note_id, note_text in notes.items():
    vector = get_embeddings(note_text)
    index_pc.upsert(vectors=[(note_id, vector,{'summary':note_text})])


{'data': [Embedding(embedding=[-0.010852213017642498, 0.022038547322154045, 0.009642699733376503, -0.0205149594694376, -0.017307410016655922, 0.007036564406007528, -0.009936724789440632, -0.00974961742758751, -0.017414327710866928, -0.03520286828279495, -0.013424936681985855, 0.03827677294611931, -0.050037793815135956, 0.01635850965976715, -0.005877168383449316, 0.005970722064375877, 0.02193162776529789, 0.013291288167238235, -0.005827050656080246, -0.0373946949839592, -0.03183494135737419, 0.003568400163203478, 0.0004815502616111189, 0.0047812554985284805, -0.014286966063082218, -0.004874808713793755, 0.01909829117357731, -0.03934595733880997, -0.016064483672380447, 0.011420217342674732, 0.0017123643774539232, -0.003859084565192461, -0.020902538672089577, 0.006314865779131651, -0.024511033669114113, 0.013371476903557777, 0.023976441472768784, -0.012616366147994995, 0.009201661683619022, -0.008520056493580341, 0.01372564397752285, 0.0023505333811044693, 0.00022970740974415094, -0.01227

{'data': [Embedding(embedding=[-0.005597195588052273, -0.014975457452237606, 0.006567827425897121, -0.011715218424797058, -0.022023526951670647, 0.01166110672056675, -0.016666453331708908, -0.018208641558885574, -0.025121431797742844, -0.03279178962111473, 0.009929526597261429, 0.021279489621520042, -0.004589362535625696, 0.023565715178847313, 0.001182851498015225, 0.02106304280459881, 0.039745163172483444, 0.009354588575661182, 0.02522965520620346, 0.001738343620672822, -0.013960860669612885, 0.017451075837016106, -0.028435783460736275, -0.031925998628139496, -0.006486659403890371, 0.040069833397865295, 0.02505379170179367, -0.03390108048915863, 0.004241017159074545, 0.001976774074137211, 0.025662550702691078, -0.000667097803670913, -0.02930157259106636, -0.007805636152625084, -0.026582451537251472, 0.014745482243597507, 0.020535452291369438, -0.0002464626159053296, 0.014529035426676273, 0.00023420290381181985, 0.0023741580080240965, 0.012107528746128082, -0.01804630644619465, -0.0216

{'data': [Embedding(embedding=[-0.010991550981998444, 0.017976773902773857, 0.01880388706922531, -0.022887755185365677, -0.011379259638488293, -0.005721938796341419, 0.008548984304070473, 0.00574132427573204, -0.024503210559487343, -0.03096502646803856, 0.0012834783410653472, 0.013188568875193596, -0.03375653177499771, -0.010687844827771187, -0.025911886245012283, -0.006810755003243685, 0.02005748078227043, 0.010694307275116444, -0.0042809536680579185, -0.01420307345688343, -0.06265377998352051, 0.01332426629960537, -0.03773701190948486, -0.03983063995838165, -0.01933375559747219, 0.020781204104423523, 0.00936963502317667, -0.043371714651584625, 0.0060062590055167675, -0.0036056938115507364, -0.003990171942859888, -0.003285833867266774, -0.01579268090426922, -0.016542252153158188, -0.012141753919422626, 0.005931947845965624, 0.022138185799121857, 0.008704067207872868, 0.011437416076660156, 0.015715138986706734, 0.019230367615818977, 0.005518391728401184, -0.0048625171184539795, -0.0180

{'data': [Embedding(embedding=[-0.00020453774777706712, 0.005470825359225273, -0.012612912803888321, -0.006361028179526329, -0.00911349430680275, -0.005085411947220564, -0.0045294612646102905, -0.0075581977143883705, -0.005757327191531658, -0.04933292791247368, 0.0012253079330548644, 0.03315238282084465, -0.0461677648127079, 0.002813859609887004, -0.004474889487028122, 0.006326920818537474, 0.025826115161180496, 0.0037245270796120167, -0.01156922709196806, -0.0423477366566658, -0.0470409132540226, -0.008676920086145401, -0.019905073568224907, 0.009986643679440022, -0.007292159833014011, -0.020068788900971413, 0.010416396893560886, -0.034653108566999435, -0.009406818076968193, 0.017231052741408348, -0.010198109783232212, -0.005030840169638395, 0.0014623543247580528, -0.002222096547484398, -0.021433083340525627, -0.0010581817477941513, 0.025962544605135918, -0.013465597294270992, 0.023261239752173424, -0.017913201823830605, 0.010204930789768696, -0.002206748118624091, -0.0052081984467804

{'data': [Embedding(embedding=[-0.022482749074697495, 0.031586192548274994, -0.023310335353016853, -0.01919999159872532, -0.011117236688733101, 0.009351720102131367, -0.009793099015951157, -0.014551717787981033, 0.0055758594535291195, -0.005255170166492462, -0.013427580706775188, 0.011537926271557808, -0.02190343849360943, -0.004962066654115915, -0.02533791959285736, 0.009944823570549488, 0.019103439524769783, -0.008496548049151897, 0.010199995711445808, -0.021255163475871086, -0.013234477490186691, -0.013255166821181774, -0.0046724118292331696, -0.0010568960569798946, -0.029820676892995834, 0.003848274238407612, 0.03412412479519844, -0.027889642864465714, -0.02078619785606861, -0.011882753111422062, 0.023558611050248146, -0.0009896547999233007, -0.015062062069773674, -0.025406885892152786, -0.023655163124203682, -0.018013784661889076, -0.013468959368765354, -0.01241378765553236, 0.03922756761312485, -0.016289647668600082, 0.005417238920927048, -0.01375171821564436, -0.0009681030060164

...

{'data': [Embedding(embedding=[-0.005064027383923531, -0.00730975391343236, 0.01973496377468109, -0.018020668998360634, -0.01570294238626957, -0.00014817937335465103, -0.026907574385404587, -0.00931205041706562, -0.044489383697509766, -0.03228360414505005, 0.027634434401988983, 0.03985393047332764, 0.015785228461027145, 0.007817184552550316, 0.012761211954057217, 0.005050312727689743, 0.046189963817596436, -0.0052525997161865234, 0.025865282863378525, -0.02003667876124382, -0.028608154505491257, -0.01444122102111578, -0.009325764141976833, -0.013104070909321308, 0.0007834327989257872, -0.0019011531258001924, 0.030199019238352776, -0.028279010206460953, 0.0016988662537187338, 0.003600019495934248, 0.01998182199895382, 0.001398007501848042, -0.017129234969615936, -0.020256109535694122, -0.019830964505672455, -0.0020845825783908367, 0.009593194350600243, -0.002130868611857295, 0.016896091401576996, -0.017211521044373512, -0.01268578227609396, -0.001830867026001215, 0.00109114870429039, -0

{'data': [Embedding(embedding=[-0.022722002118825912, -0.01277528889477253, 0.009119488298892975, -0.016290994361042976, -0.03783886879682541, 0.014302986674010754, -0.015370373614132404, -0.002026367699727416, -0.03847930207848549, -0.06292246282100677, 0.005677164997905493, 0.02927308902144432, -0.0052802301943302155, 0.004966685548424721, -0.0012083154870197177, -0.016584526747465134, 0.02567065879702568, -0.021974829956889153, 0.035784151405096054, -0.022962164133787155, -0.014289643615484238, -0.0127819599583745, -0.0103403115645051, -0.02173466794192791, -0.008018745109438896, 0.0228954516351223, 0.010620500892400742, -0.03618441894650459, -0.007558434270322323, 0.0024566580541431904, 0.0029770091641694307, 0.0013617523945868015, -0.00729158753529191, -0.026564594358205795, -0.002553389873355627, 0.02405623532831669, 0.008185524493455887, -0.010620500892400742, 0.01139435637742281, -0.0001326937199337408, -0.010026766918599606, 0.011421041563153267, -0.018959462642669678, -0.0341

{'data': [Embedding(embedding=[0.011202913708984852, 0.019660916179418564, -0.003360545728355646, -0.0378246083855629, -0.028552325442433357, 0.01484090555459261, 0.002682526595890522, -0.008812608197331429, -0.014000358991324902, -0.024047520011663437, -0.010841741226613522, 0.028289655223488808, -0.04273655638098717, -0.008018028922379017, -0.008090263232588768, 0.005923228804022074, 0.03209838271141052, -0.008812608197331429, -0.01703420840203762, -0.028053252026438713, -0.05122082307934761, 0.01183332409709692, 0.009462718851864338, 0.012549102306365967, 0.0048265778459608555, -0.003920363262295723, 0.02528207376599312, -0.028289655223488808, 0.003746343543753028, 0.010782640427350998, -0.007321950979530811, -0.011012476868927479, -0.0013018626486882567, -0.022812968119978905, -0.012746104970574379, -0.0006640648352913558, 0.016745269298553467, -0.012076294049620628, 0.004081249237060547, -0.009298549965023994, 0.011386782862246037, -0.011143812909722328, -0.004169900435954332, -0.

{'data': [Embedding(embedding=[-0.013520505279302597, 0.0015584143111482263, 0.025618519634008408, -0.03506987914443016, -0.0012301472015678883, -0.0058609372936189175, -0.011065340600907803, -0.018054695799946785, -0.010784945450723171, -0.038762886077165604, 0.023758338764309883, 0.0332644097507, -0.015579014085233212, 0.011810780502855778, 0.004534190986305475, -0.007167167495936155, 0.030063804239034653, -0.018711229786276817, 0.02489359676837921, -0.023471105843782425, -0.016878405585885048, 0.0034468057565391064, -0.032361675053834915, -0.005272792186588049, -0.012214276008307934, -0.0024585844948887825, 0.012645126320421696, -0.027683867141604424, -0.0034006431233137846, -0.006308885291218758, 0.01683737151324749, -0.014594213105738163, -0.008500752970576286, -0.00906838197261095, -0.023129159584641457, 0.010408805683255196, 0.005259114317595959, -0.015907282009720802, 0.02419602870941162, 0.003279252676293254, -0.0024705526884645224, 0.0009394938824698329, -0.0072629121132195, 

{'data': [Embedding(embedding=[-0.0013016491429880261, 0.0321313701570034, 0.003354281885549426, -0.006960009224712849, -0.0321313701570034, 0.005156307481229305, 0.018975747749209404, 0.01105689350515604, -0.023414596915245056, -0.038273345679044724, 0.000583772431127727, 0.0016930658603087068, -0.02598939836025238, 0.003925900906324387, 0.01037296187132597, 0.01835886761546135, 0.0243265051394701, -0.013128803111612797, 0.006406829226762056, -0.02939564548432827, -0.05444631353020668, -0.0038420860655605793, -0.018479561433196068, -0.004180699121206999, -0.013973659835755825, -0.008575964719057083, 0.015073314309120178, -0.02802778221666813, -0.014751464128494263, -0.003490062430500984, -0.013115392997860909, -0.01375909335911274, -0.0075768884271383286, -0.014362562447786331, -0.005159659776836634, 0.007422668859362602, 0.023360956460237503, -0.024125350639224052, 0.005612261593341827, -0.023615753278136253, 0.008736889809370041, 0.011921865865588188, -0.0024272864684462547, 0.00613